![CH6-ADS.png](.\Media\CH6-ADS.png)

# <span style="color:#cc5500;">SQL Server Ports and Protocols</span>

Securing SQL Server is a big priority for DBA's and many times I get asked questions related to the ports that the various SQL Server features use so that our customers can apply for the appropriate firewall exception rules.  Many MCS consultants from other technologies frequently ask me for SQL Port information and SQL Books On-Line has good documentation on this, but it can be difficult to find what you are looking for if you are not a regular reader of SQL BOL.  I have consolidated all of the various bits of information into this single notebook so that the appropriate firewall exceptions can be requested.

Firewalled systems help prevent unauthorized access to computer resources.  To access an instance of the SQL Server Database Engine through a firewall, you must configure the firewall on the computer running SQL Server to allow access.  Opening ports in your firewall can leave your server exposed to malicious attacks.  Make sure that you understand firewall systems before you open ports. For more information, see [Security Considerations - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/sql-server/install/security-considerations-for-a-sql-server-installation?redirectedfrom=MSDN&view=sql-server-ver15) 

Enable only those protocols that are needed.  For example, if TCP/IP is sufficient, there is no need to enable the Named Pipes protocol.  When SQL Server is installed, no Service Broker, HTTP, or Database Mirroring endpoints are created.  This is in line with the general policy of “off by default, enable when needed”.

## <span style="color:#cc5500;">Ports used for the Core Database Engine</span>

- SQL Server default instance running over TCP 
    - TCP port 1433
    - This is the most common port allowed through the firewall.  It is also the port that will be used in for a default instance installation of SQL Server.  It applies to routine connections to the default installation of the Database Engine, or a named instance that is the only instance running on the computer
- SQL Server named instances in the standard configuration 
    - The TCP port is a Dynamic Port determined at the time the Database Engine starts.  
    - See my note below in the Recommendation and Best Practices section for more information on Dynamic ports and what we recommend you do.
- SQL Server named instances when they are configured to use a fixed port 
    - The port number configured by the administrator.
- Dedicated Administrators Connection. The (DAC)
    - TCP port 1434 
    - By default, remote connections to the Dedicated Administrator Connection (DAC) are not enabled.
- SQL Server Browser service
    - UDP port 1434
    - The SQL Server Browser service listens for incoming connections to a named instance and provides the client the TCP port number that corresponds to that named instance. Normally the SQL Server Browser service is started whenever named instances of the Database Engine are used. The SQL Server Browser service does not have to be started if the client is configured to connect to the specific port of the named instance.
- Transact-SQL debugger
    - TCP port 135
    - If using Visual Studio, on the Visual Studio host computer, you must also add Devenv.exe to the Exceptions list and open TCP port 135
    - If using Management Studio, on the Management Studio host computer, you must also add ssms.exe to the Exceptions list and open TCP port 135
- Replication
    - TCP 21
- Cluster Admin Center, File and Print Sharing
    - TCP 137 and 138
- File Stream
    - TCP 139 and 445
- HTTPS
    - TCP 443
- iSCSI
    - TCP 860 and 3260

## <span style="color:#cc5500;">Ports used for Analysis Services</span>

- Analysis Services
    - TCP port 2383 and 2725 is the standard port for the default instance
- SQL Server Browser service for Analysis Services
    - TCP port 2382
    - This is only needed for an Analysis Services named instance. 
    - Client connection requests for a named instance of Analysis Services that do not specify a port number are directed to port 2382, the port on which SQL Server Browser listens. SQL Server Browser then redirects the request to the port that the named instance uses.
- Analysis Services configured for use through IIS/HTTP
    - TCP port 80
    - Used for an HTTP connection through a URL.

## <span style="color:#cc5500;">Ports used for Reporting Services</span>

- Reporting Services Web Services
    - TCP port 80
    - Used for an HTTP connection to Reporting Services through a URL. 
    - Microsoft recommends that you do not use the preconfigured Windows Firewall rule “World Wide Web Services” (HTTP)  If using Windows Firewall rules, we recommend that you do not use the preconfigured rule.  For example, the rule groups World Wide Web Services (HTTP) and World Wide Web Services (HTTPS) are associated with IIS. Enabling those rules will open ports 80 and 443, and SQL Server features that depend on ports 80 and 443 will function if those rules are enabled. However, administrators configuring IIS might modify or disable those rules. Therefore, if you are using port 80 or port 443 for SQL Server, you should create your own rule or rule group that maintains your desired port configuration independently of the other IIS rules.
- Reporting Services configured for use through HTTPS
    - TCP port 443
    - Used for an HTTPS connection through a URL. HTTPS is an HTTP connection that uses secure sockets layer (SSL).

## <span style="color:#cc5500;">Ports used for Power BI Report Server</span>

- Power BI Report Server
    - TCP 80 is the standard port used
    - TCP 443 is the standard port for https

## <span style="color:#cc5500;">Ports used for Integration Services</span>

- SSIS
    - TCP 3882
- Microsoft remote procedure calls (MS RPC) Used by the Integration Services runtime
    - TCP port 135
    - The Integration Services service uses DCOM on port 135. The Service Control Manager uses port 135 to perform tasks such as starting and stopping the Integration Services service and transmitting control requests to the running service. The port number cannot be changed.

## <span style="color:#cc5500;">Ports used for Service Broker</span>

- Service Broker
    - TCP port 4022
    - There is no default port for SQL Server Service Broker, but this is the conventional configuration 4022 is used in Books Online examples

## <span style="color:#cc5500;">Ports used for Database Mirroring</span>

- Database Mirroring
- This port is chosen by the DBA.  There is no default port for Database mirroring however Books online examples use TCP port 7022 so we see that port commonly used. It is very important to avoid interrupting an in-use mirroring endpoint, especially in high-safety mode with automatic failover. Your firewall configuration must avoid breaking quorum so choose a port, apply for the exception and coordinate with the DBA if this port ever needs to change

## <span style="color:#cc5500;">Ports used for Always On Availability Groups</span>

- Default Port
    - TCP 5022
    - And whatever static ports you decide to use

## <span style="color:#cc5500;">Ports used for Addional Services</span>

- The Windows Clustering Service
    - TCP 3343 for Node Join operations and UDP 3343 (Unicast) for other cluster communications
    - TCP 445 for Cluster communications
    - UDP 5000-5099
    - UDP 8011-8031 Clustering Internode RPC
    - MSDTC and WMI use port 135
- PowerShell
    - TCP 9389
- RDP
    - TCP 3389
- IPSEC
    - UDP 500 and 4500

## <span style="color:#cc5500;">Recommendations and Practices</span>

- CHANGE THE STANDARD PORT FOR A DEFAULT INSTANCE
    - The default port for a SQL Server database engine install as I allude to above, out of the box, SQL Server listens on TCP port 1433.  This is a known fact to the world.  It is recommended that you change SQL Server to listen on another static port.  (For example port 50007)
- LIMIT THE USE OF DYNAMIC PORTS: HOW DOES THIS WORK?
    - By default Named instances (including SQL Server Express) use dynamic ports. 
    - That means that every time that the Database Engine starts-up, it identifies an available port and uses that port number.  If the named instance is the only instance of the Database Engine installed, it will probably use TCP port 1433.  If other instances of the Database Engine are installed, it will probably use a different TCP port. 
    - Because the port selected might change every time that the Database Engine is started, it is difficult to configure the firewall to enable access to the correct port number. Therefore, if a firewall is started, we recommend reconfiguring the Database Engine to use the same port number every time. 
    - This is called a fixed port or a static port.
- THE FIRE WALL EXCEPTION FOR A PROGRAM NAME
    - There have been some recommendations published on the internet about using a program name like (sqlserver.exe) as an exception to the Windows firewall rule instead of using a port strategy.
    - One alternative to a port strategy would be configuring a named instance to listen on a fixed port and then to create an exception in the firewall for a SQL Server program name such as sqlservr.exe (for the Database Engine).  This can be convenient, but the port number will not appear in the Local Port column of the Inbound Rules page when you are using the Windows Firewall with Advanced Security MMC snap-in. 
    - This can make it more difficult to audit which ports are open.
    - Another consideration is that a service pack or cumulative update can change the path to the SQL Server executable which will invalidate the firewall rule.
    - I recommend a port strategy
- NAMED PIPES
    - Named Pipes are enabled by default
    - It is a Microsoft Best Practice to disable Named Pipes if they are not necessary for client connections. 
    - By default, SQL Server listens on: \\\\.\\pipe\\sql\\query for the default instance and \\\\.\\pipe\\MSSQL$\<instancename\>\\sql\\query for a named instance.
    - Since sql\\query path is well known as the named pipe used by SQL Server, if named pipes need to be enabled and used, then changing the pipe can help reduce the risk of attack by malicious programs
    - For example: you could change it to something like: \\\\.\\pipe\\unit\\app to listen on the unit\\app pipe.
- THE DEDICATED ADMINISTRATOR CONNECTION (DAC)
    - SQL Server provides a special diagnostic connection for administrators when standard connections to the server are not possible.  This diagnostic connection allows an administrator to access SQL Server to execute diagnostic queries and troubleshoot problems even when SQL Server is not responding to standard connection requests
    - As mentioned above the DAC uses TCP 1434.  By default, network connections are not permitted unless they are enabled first.
    - For SQL 2012 and later, use sp\_configure to enable

## <span style="color:#cc5500;">Helpful Links and Resources</span>

[SQL Server Ports - Microsoft Tech Community](https://techcommunity.microsoft.com/t5/sql-server-blog/sql-server-ports/ba-p/384096)

[Configure Windows Firewall - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/sql-server/install/configure-the-windows-firewall-to-allow-sql-server-access?redirectedfrom=MSDN&view=sql-server-ver15)

[Configure a Windows Firewall for Database Engine Access - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/database-engine/configure-windows/configure-a-windows-firewall-for-database-engine-access?view=sql-server-ver15)

[Configure the Windows Firewall to Allow Analysis Services Access | Microsoft Docs](https://docs.microsoft.com/en-us/analysis-services/instances/configure-the-windows-firewall-to-allow-analysis-services-access?view=asallproducts-allversions)

[Configure a Firewall for Report Server Access - SQL Server Reporting Services (SSRS) | Microsoft Docs](https://docs.microsoft.com/en-us/sql/reporting-services/report-server/configure-a-firewall-for-report-server-access?view=sql-server-ver15)

In [ ]:
--Power Shell Commands that your Administrator can run to open up ports....this is a sample of the many listed above

New-NetFirewallRule -DisplayName "SQLServer default instance" -Direction Inbound -LocalPort 1433 -Protocol TCP -Action Allow
New-NetFirewallRule -DisplayName "SQLServer Browser service" -Direction Inbound -LocalPort 1434 -Protocol UDP -Action Allow